In [81]:
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker
import os
import sys
import pandas as pd
from tqdm import tqdm


def calc_distance(lat1, lon1, lat2, lon2, distance):
    x, y = abs(lon1 - lon2) * 88800, abs(lat1 - lat2) * 111320
    z = (x**2 + y**2) ** (1 / 2)
    return distance > z


def tagging(buildings, label):
    new_tags = [
        Tag(
            building_id=building.id,
            label=label,
        )
        for building in buildings
    ]
    session.add_all(new_tags)


HOSPITAL_DISTANCE = 3000
SUBWAY_STATION_DISTANCE = 500
BUS_STATION_DISTANCE = 50

sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "../")))

from scripts.models import Address, Hospital, BusStation, Subway, Building, Tag

engine = create_engine(os.getenv("DATABASE_URL"), echo=False)
Session = sessionmaker(bind=engine)
session = Session()

In [ ]:
results = session.query(Address).all()
address_df = pd.DataFrame(row.__dict__ for row in results)
address_df = address_df[["latitude", "longitude", "id"]]
address_df

In [ ]:
results = session.query(Hospital).all()
hospital_df = pd.DataFrame(row.__dict__ for row in results)
hospital_df = hospital_df[["latitude", "longitude"]]
hospital_df

In [ ]:
results = session.query(BusStation).all()
bus_station_df = pd.DataFrame(row.__dict__ for row in results)
bus_station_df = bus_station_df[["latitude", "longitude"]]
bus_station_df

In [ ]:
results = session.query(Subway).all()
subway_station_df = pd.DataFrame(row.__dict__ for row in results)
subway_station_df = subway_station_df[["latitude", "longitude"]]
subway_station_df

In [ ]:
for data in tqdm(address_df.itertuples()):
    buildings = session.query(Building).filter_by(address_id=data.id).all()
    if any(
        calc_distance(
            data.latitude,
            data.longitude,
            row.latitude,
            row.longitude,
            SUBWAY_STATION_DISTANCE,
        )
        for row in subway_station_df.itertuples()
    ):
        tagging(buildings, "역세권")
    if any(
        calc_distance(
            data.latitude,
            data.longitude,
            row.latitude,
            row.longitude,
            BUS_STATION_DISTANCE,
        )
        for row in bus_station_df.itertuples()
    ):
        tagging(buildings, "버스정류장 가까움")
    if any(
        calc_distance(
            data.latitude,
            data.longitude,
            row.latitude,
            row.longitude,
            HOSPITAL_DISTANCE,
        )
        for row in hospital_df.itertuples()
    ):
        tagging(buildings, "병원 가까움")


session.commit()
session.close()